In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('all_matches.csv')

In [35]:
df.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
dtype: object

In [36]:
df['ball'] = None

# We will use a variable to keep track of the current over and ball
current_over = 0
current_ball = 1

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    wides = row['wides']  # Assuming 'wides' column exists and records the number of wides
    noballs = row['noballs']  # Assuming 'noballs' column exists and records the number of no-balls

    # Set the 'ball' column with the current over and ball
    df.at[i, 'ball'] = f"{current_over}.{current_ball}"

    # If the delivery is legal, increment the balls
    if pd.isna(wides) and pd.isna(noballs):  # Assuming NaN values indicate no wides or no-balls
        if current_ball == 6:
            current_over += 1
            current_ball = 1
        else:
            current_ball += 1
    # If the delivery is a wide or no-ball, do not increment the balls

In [38]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [39]:
innings1_runs = df[df['innings'] == 1].groupby('match_id')['runs_off_bat'].sum()

# Map the sum to innings 2 rows based on match_id
df['target'] = df.apply(lambda x: innings1_runs.get(x['match_id']) if x['innings'] == 2 else None, axis=1)

In [40]:
df['target'] = df.apply(lambda x: innings1_runs.get(x['match_id']) if x['innings'] == 2 else 0, axis=1)

In [43]:
# Assume df has a column 'overs' with string representations of over and ball, like '0.1', '0.2', ..., '19.6'

# Create 'over' and 'ball' columns from 'overs'
df['over'] = df['overs'].apply(lambda x: int(x.split('.')[0]))
df['ball'] = df['overs'].apply(lambda x: int(x.split('.')[1]))

In [44]:
# Create new column 'isPlayerOut' 
df['isPlayerOut'] = 0
# Check for null, update as 0  
df.loc[df['player_dismissed'].isna(), 'isPlayerOut'] = 0
# Else update as 1
df.loc[df['player_dismissed'].notna(), 'isPlayerOut'] = 1

In [45]:
# Initialize new columns
df['striker_total_runs'] = 0
df['non_striker_total_runs'] = 0

In [46]:
# Initialize a dictionary to keep track of the players' total runs
player_runs = {}

# Initialize columns for striker_total_runs and non_striker_total_runs
df['striker_total_runs'] = 0
df['non_striker_total_runs'] = 0

# Process each row in the DataFrame
for i, row in df.iterrows():
    striker = row['striker']
    non_striker = row['non_striker']
    runs_off_bat = row['runs_off_bat']
    isPlayerOut = row['isPlayerOut']

    # Update the total runs for the striker and non-striker
    player_runs[striker] = player_runs.get(striker, 0) + runs_off_bat
    player_runs[non_striker] = player_runs.get(non_striker, 0)

    # Assign the total runs to the dataframe
    df.at[i, 'striker_total_runs'] = player_runs[striker]
    df.at[i, 'non_striker_total_runs'] = player_runs[non_striker]

    # If a player is out, reset their score
    if isPlayerOut:
        # Check the next row to see if striker or non_striker has changed
        if i < len(df) - 1: 
            next_row = df.iloc[i + 1]
            if next_row['striker'] != striker:
                player_runs[striker] = 0
            if next_row['non_striker'] != non_striker:
                player_runs[non_striker] = 0

In [47]:
# Ensure that 'extras' column exists and is integer
if 'extras' not in df.columns:
    df['extras'] = 0
else:
    df['extras'] = df['extras'].fillna(0).astype(int)

# Initialize columns for bowler_wickets and bowler_runs
df['bowler_wickets'] = 0
df['bowler_runs'] = 0

# Initialize a dictionary to keep track of the bowlers' stats
bowler_stats = {}

# Process each row in the DataFrame
for i, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    bowler = row['bowler']
    runs_conceded = row['runs_off_bat'] + row['extras']
    is_wicket = row['isPlayerOut']

    # Initialize the bowler's stats if not already present
    if (match_id, innings, bowler) not in bowler_stats:
        bowler_stats[(match_id, innings, bowler)] = {'wickets': 0, 'runs': 0}

    # Update bowler stats
    bowler_stats[(match_id, innings, bowler)]['runs'] += runs_conceded
    if is_wicket:
        bowler_stats[(match_id, innings, bowler)]['wickets'] += 1

    # Assign the running totals to the DataFrame
    df.at[i, 'bowler_wickets'] = bowler_stats[(match_id, innings, bowler)]['wickets']
    df.at[i, 'bowler_runs'] = bowler_stats[(match_id, innings, bowler)]['runs']


In [48]:
# Calculate the team score for each ball and then the cumulative sum within each group
df['team_score'] = (df['runs_off_bat'] + df['extras']).groupby([df['match_id'], df['innings']]).cumsum()

In [49]:
# Calculate the total wickets fallen per ball as a cumulative sum within each group
df['team_wickets'] = df.groupby(['match_id', 'innings'])['isPlayerOut'].cumsum()

In [50]:
# Initialize the new columns with zeros
df['striker_balls'] = 0
df['non_striker_balls'] = 0

# We will use a dictionary to keep track of the balls faced by each player
player_balls = {}

# Initialize variables to store the current striker and non-striker
current_striker = None
current_non_striker = None

# Function to swap the players and their ball counts
def swap_balls():
    global current_striker, current_non_striker
    current_striker, current_non_striker = current_non_striker, current_striker

# Iterate over the DataFrame rows
for i in range(len(df)):
    # Get current row details
    row = df.iloc[i]
    
    # If it's the first ball or new players are in, initialize them
    if current_striker != row['striker'] or current_striker is None:
        if current_striker is not None:
            # Swap the players if the striker has changed, not if it's the first ball
            swap_balls()
        current_striker = row['striker']
        current_non_striker = row['non_striker']
        if current_striker not in player_balls:
            player_balls[current_striker] = 0
        if current_non_striker not in player_balls:
            player_balls[current_non_striker] = 0
    
    # Increment the ball count for the current striker
    player_balls[current_striker] += 1
    
    # Update the DataFrame with the current ball count for striker and non-striker
    df.at[i, 'striker_balls'] = player_balls[current_striker]
    df.at[i, 'non_striker_balls'] = player_balls[current_non_striker]

    # If the player is out, check who will be out in the next delivery
    if row['isPlayerOut'] == 1 and i < len(df) - 1:
        next_row = df.iloc[i + 1]
        # If the striker in the next delivery is different, the current striker got out
        if current_striker != next_row['striker']:
            player_balls[current_striker] = 0
        # If the non-striker in the next delivery is different, the current non-striker got out
        elif current_non_striker != next_row['non_striker']:
            player_balls[current_non_striker] = 0

In [51]:
df['batsman_4s'] = 0
df['batsman_6s'] = 0

# We will use a dictionary to keep track of the 4s and 6s scored by each player
player_4s = {}
player_6s = {}

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    striker = row['striker']
    runs_off_bat = row['runs_off_bat']

    # Initialize the player in the dictionaries if they're not already present
    if striker not in player_4s:
        player_4s[striker] = 0
    if striker not in player_6s:
        player_6s[striker] = 0

    # Update the count if a boundary is hit
    if runs_off_bat == 4:
        player_4s[striker] += 1
    elif runs_off_bat == 6:
        player_6s[striker] += 1

    # Update the DataFrame with the current 4s and 6s for the striker
    df.at[i, 'batsman_4s'] = player_4s[striker]
    df.at[i, 'batsman_6s'] = player_6s[striker]

In [52]:
df.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
overs                      object
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int32
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
target                      int64
over                        int64
ball                        int64
isPlayerOut                 int64
striker_total_runs          int64
non_striker_total_runs      int64
bowler_wickets              int64
bowler_runs   

In [53]:
df['bowler_balls_bowled'] = 0

# We will use a dictionary to keep track of the balls bowled by each bowler in each match and innings
bowler_balls = {}

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    bowler = row['bowler']

    # Initialize the bowler in the dictionary if they're not already present for the match and innings
    if (match_id, innings, bowler) not in bowler_balls:
        bowler_balls[(match_id, innings, bowler)] = 0

    # Increment the ball count for the bowler for the specific match and innings
    bowler_balls[(match_id, innings, bowler)] += 1

    # Update the DataFrame with the current ball count for the bowler
    df.at[i, 'bowler_balls_bowled'] = bowler_balls[(match_id, innings, bowler)]

In [54]:
# Display the DataFrame to verify the results
print(df[['match_id', 'innings', 'bowler', 'bowler_balls_bowled']])

       match_id  innings            bowler  bowler_balls_bowled
0        682897        1  Mashrafe Mortaza                    1
1        682897        1  Mashrafe Mortaza                    2
2        682897        1  Mashrafe Mortaza                    3
3        682897        1  Mashrafe Mortaza                    4
4        682897        1  Mashrafe Mortaza                    5
...         ...      ...               ...                  ...
35256   1298179        2    Mohammad Wasim                   22
35257   1298179        2    Mohammad Wasim                   23
35258   1298179        2    Mohammad Wasim                   24
35259   1298179        2    Mohammad Wasim                   25
35260   1298179        2    Mohammad Wasim                   26

[35261 rows x 4 columns]


In [55]:
import pandas as pd

# Assuming 'df' is your DataFrame with the necessary columns

# Initialize the new column with zeros
df['team_score'] = 0

# We will use a dictionary to keep track of the team score for each match and innings
team_scores = {}

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    runs_off_bat = row['runs_off_bat']
    extras = row['extras']  # Assuming 'extras' column exists and has the extra runs

    # Initialize the team score in the dictionary if they're not already present for the match and innings
    if (match_id, innings) not in team_scores:
        team_scores[(match_id, innings)] = 0

    # Increment the team score for the specific match and innings
    team_scores[(match_id, innings)] += runs_off_bat + extras

    # Update the DataFrame with the current team score
    df.at[i, 'team_score'] = team_scores[(match_id, innings)]

# Display the DataFrame to verify the results
print(df[['match_id', 'innings', 'runs_off_bat', 'extras', 'team_score']])

       match_id  innings  runs_off_bat  extras  team_score
0        682897        1             0       0           0
1        682897        1             0       1           1
2        682897        1             0       0           1
3        682897        1             0       0           1
4        682897        1             0       0           1
...         ...      ...           ...     ...         ...
35256   1298179        2             0       0         132
35257   1298179        2             1       0         133
35258   1298179        2             4       0         137
35259   1298179        2             0       0         137
35260   1298179        2             1       0         138

[35261 rows x 5 columns]


In [56]:
# Initialize the new column with tens
df['remaining_wickets'] = 10

# We will use a dictionary to keep track of the remaining wickets for each match and innings
wickets_remaining = {}

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    is_player_out = row['isPlayerOut']  # Assuming 'isPlayerOut' column is 1 if a player is out

    # Initialize the wickets remaining in the dictionary if they're not already present for the match and innings
    if (match_id, innings) not in wickets_remaining:
        wickets_remaining[(match_id, innings)] = 10

    # Decrement the wickets remaining if a player is out
    if is_player_out == 1:
        wickets_remaining[(match_id, innings)] -= 1

    # Update the DataFrame with the current number of remaining wickets
    df.at[i, 'remaining_wickets'] = wickets_remaining[(match_id, innings)]

# Display the DataFrame to verify the results
print(df[['match_id', 'innings', 'isPlayerOut', 'remaining_wickets']])

       match_id  innings  isPlayerOut  remaining_wickets
0        682897        1            1                  9
1        682897        1            0                  9
2        682897        1            0                  9
3        682897        1            0                  9
4        682897        1            0                  9
...         ...      ...          ...                ...
35256   1298179        2            1                  5
35257   1298179        2            0                  5
35258   1298179        2            0                  5
35259   1298179        2            0                  5
35260   1298179        2            0                  5

[35261 rows x 4 columns]


In [58]:
df['remaining_balls'] = 120

# We will use a dictionary to keep track of the remaining balls for each match and innings
balls_remaining = {}

# Iterate over the DataFrame rows
for i, row in df.iterrows():
    match_id = row['match_id']
    innings = row['innings']
    wides = row['wides']  # Assuming 'wides' column exists and records the number of wides
    noballs = row['noballs']  # Assuming 'noballs' column exists and records the number of no-balls

    # Initialize the balls remaining in the dictionary if they're not already present for the match and innings
    if (match_id, innings) not in balls_remaining:
        balls_remaining[(match_id, innings)] = 120

    # Decrement the balls remaining if it's not a wide or no-ball
    if pd.isna(wides) and pd.isna(noballs):  # Assuming NaN values indicate no wides or no-balls
        balls_remaining[(match_id, innings)] -= 1

    # Update the DataFrame with the current number of remaining balls
    df.at[i, 'remaining_balls'] = balls_remaining[(match_id, innings)]

In [59]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'overs',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'target', 'over', 'ball', 'isPlayerOut',
       'striker_total_runs', 'non_striker_total_runs', 'bowler_wickets',
       'bowler_runs', 'team_score', 'team_wickets', 'striker_balls',
       'non_striker_balls', 'batsman_4s', 'batsman_6s', 'bowler_balls_bowled',
       'remaining_wickets', 'remaining_balls'],
      dtype='object')

In [63]:
# Initialize the runs_to_win column with zeros
df['runs_to_win'] = 0

# Define a function to calculate runs to win for the second innings
def calculate_runs_to_win(row):
    if row['innings'] == 2:
        return max(row['target'] - row['team_score'], 0)
    else:
        return 0

# Apply the function to each row in the dataframe
df['runs_to_win'] = df.apply(calculate_runs_to_win, axis=1)

# Convert runs_to_win column to an integer type
df['runs_to_win'] = df['runs_to_win'].astype(int)

In [64]:
df.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
overs                      object
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int32
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
target                      int64
over                        int64
ball                        int64
isPlayerOut                 int64
striker_total_runs          int64
non_striker_total_runs      int64
bowler_wickets              int64
bowler_runs   

In [66]:
df.to_csv('aa.csv', index=False)